# Kerberoasting

**Kerberoasting** is a post-exploitation attack technique that targets Active Directory environments by abusing the Kerberos authentication protocol.

## The Attack Process

1. **Request Service Tickets**: An attacker with valid domain credentials requests Kerberos service tickets (TGS) for Service Principal Names (SPNs) registered in Active Directory

2. **Extract Encrypted Material**: The service tickets contain a portion encrypted with the service account's password hash (typically using RC4 or AES encryption)

3. **Offline Cracking**: The attacker extracts these tickets and performs offline password cracking to recover the plaintext password of the service account

4. **Privilege Escalation**: If the service account has elevated privileges (like Domain Admin), the attacker gains high-level access to the domain

## Why It's Effective

- **No special privileges required**: Any authenticated domain user can request service tickets
- **Difficult to detect**: The requests look like legitimate Kerberos activity
- **Offline attack**: Password cracking happens outside the network, leaving no authentication failures
- **Weak passwords**: Service accounts often have weak, static passwords that never expire

## Detection Indicators

Classic Kerberoasting signatures to monitor:

- **Event ID 4769**: Kerberos Service Ticket Request
- **RC4 encryption (0x17)**: Attackers often downgrade to RC4 because it's faster to crack
- **Suspicious ticket options**: `0x40810010`, `0x40810000`, `0x40800010` indicate forwardable/renewable tickets
- **Excluding machine accounts (*$)**: Focuses on user/service accounts

### Example Detection Query
```kql
input.type:winlog AND 
event.code:4769 AND 
winlog.event_data.TicketEncryptionType:0x17 AND 
winlog.event_data.TicketOptions:(0x40810010 OR 0x40810000 OR 0x40800010) AND 
NOT service.name:*$
```

## Common Tools

- **Rubeus** (Windows)
- **Impacket's GetUserSPNs.py** (Linux)
- **Invoke-Kerberoast** (PowerShell)
- **Mimikatz** (Windows)

## Mitigation Strategies

1. **Use strong passwords** for service accounts (25+ characters)
2. **Implement Group Managed Service Accounts (gMSA)** - passwords rotate automatically
3. **Disable RC4 encryption** in the domain (enforce AES)
4. **Monitor for suspicious ticket requests** - especially bulk requests or requests for sensitive SPNs
5. **Audit service accounts** regularly and remove unnecessary SPNs
6. **Enable Advanced Threat Analytics (ATA)** or similar solutions

## References

- [MITRE ATT&CK: T1558.003](https://attack.mitre.org/techniques/T1558/003/)
- [Microsoft Security: Kerberoasting](https://docs.microsoft.com/en-us/security/compass/kerberoasting)

## Sigma Rule

## Elastic SIEM Query If running from Kibana

input.type:winlog AND event.code:4769 AND winlog.event_data.TicketEncryptionType:0x17 AND winlog.event_data.TicketOptions:(0x40810010 OR 0x40810000 OR 0x40800010) AND NOT service.name:*$

## Automated API Query for Event Elastic
- Make sure you are running Jupyter from venv

### Install Elastic and Tabulate

In [13]:
!pip3 install elasticsearch tabulate

### Return Elastic Results

In [15]:
from elasticsearch import Elasticsearch
import json
import warnings
from tabulate import tabulate
warnings.filterwarnings('ignore')

# Connection details
ES_ENDPOINT = "https://192.168.136.200:9200"
API_KEY = "NmpObkhac0ItSjRQNDFsakNsSm06R0lValN1US0tNm1sQVZJYnBPMXRQUQ=="

# Index pattern
INDEX_PATTERN = "logs-*"

try:
    # Initialize Elasticsearch client
    client = Elasticsearch(
        [ES_ENDPOINT],
        api_key=API_KEY,
        verify_certs=False,
        ssl_show_warn=False
    )
    
    # Test connection
    if client.ping():
        print("✓ Successfully connected to Elasticsearch")
        info = client.info()
        print(f"✓ Cluster: {info['cluster_name']}")
        print(f"✓ Version: {info['version']['number']}\n")
    else:
        print("✗ Could not connect")
        exit(1)
    
    # Step 1: Test basic query for event.code 4769
    print("="*80)
    print("STEP 1: Testing basic query for event.code:4769")
    print("="*80)
    
    basic_query = {
        "query": {
            "term": {
                "event.code": "4769"  # Try as string first
            }
        },
        "size": 1
    }
    
    response = client.search(index=INDEX_PATTERN, body=basic_query)
    print(f"Results with event.code as string '4769': {response['hits']['total']['value']}")
    
    # Try as integer
    basic_query["query"]["term"]["event.code"] = 4769
    response = client.search(index=INDEX_PATTERN, body=basic_query)
    print(f"Results with event.code as integer 4769: {response['hits']['total']['value']}")
    
    if response['hits']['total']['value'] > 0:
        print("\n✓ Found Event 4769 records! Here's a sample:")
        sample = response['hits']['hits'][0]['_source']
        
        # Print relevant fields to see their structure
        print("\n--- SAMPLE RECORD STRUCTURE ---")
        print(f"input.type: {sample.get('input', {}).get('type')}")
        print(f"event.code: {sample.get('event', {}).get('code')}")
        print(f"winlog.event_data.TicketEncryptionType: {sample.get('winlog', {}).get('event_data', {}).get('TicketEncryptionType')}")
        print(f"winlog.event_data.TicketOptions: {sample.get('winlog', {}).get('event_data', {}).get('TicketOptions')}")
        print(f"service.name: {sample.get('service', {}).get('name')}")
        
        print("\n--- FULL WINLOG EVENT DATA ---")
        print(json.dumps(sample.get('winlog', {}).get('event_data', {}), indent=2))
    else:
        print("❌ No Event 4769 records found. Check if data exists in index.")
        exit(1)
    
    # Step 2: Try the full query without .keyword
    print("\n" + "="*80)
    print("STEP 2: Testing full query WITHOUT .keyword suffixes")
    print("="*80)
    
    query_no_keyword = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"input.type": "winlog"}},
                    {"term": {"event.code": "4769"}},
                    {"term": {"winlog.event_data.TicketEncryptionType": "0x17"}},
                    {"terms": {"winlog.event_data.TicketOptions": ["0x40810010", "0x40810000", "0x40800010"]}}
                ],
                "must_not": [
                    {"wildcard": {"service.name": "*$"}}
                ]
            }
        }
    }
    
    response = client.search(index=INDEX_PATTERN, body=query_no_keyword)
    print(f"Results: {response['hits']['total']['value']}")
    
    # Step 3: Try with .keyword
    print("\n" + "="*80)
    print("STEP 3: Testing full query WITH .keyword suffixes")
    print("="*80)
    
    query_with_keyword = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"input.type.keyword": "winlog"}},
                    {"term": {"event.code": "4769"}},
                    {"term": {"winlog.event_data.TicketEncryptionType.keyword": "0x17"}},
                    {"terms": {"winlog.event_data.TicketOptions.keyword": ["0x40810010", "0x40810000", "0x40800010"]}}
                ],
                "must_not": [
                    {"wildcard": {"service.name.keyword": "*$"}}
                ]
            }
        }
    }
    
    response = client.search(index=INDEX_PATTERN, body=query_with_keyword)
    print(f"Results: {response['hits']['total']['value']}")
    
    # Step 4: Try query_string (like KQL)
    print("\n" + "="*80)
    print("STEP 4: Testing with query_string (KQL-like)")
    print("="*80)
    
    query_string = {
        "query": {
            "query_string": {
                "query": 'input.type:winlog AND event.code:4769 AND winlog.event_data.TicketEncryptionType:0x17 AND winlog.event_data.TicketOptions:(0x40810010 OR 0x40810000 OR 0x40800010) AND NOT service.name:*$'
            }
        }
    }
    
    response = client.search(index=INDEX_PATTERN, body=query_string)
    print(f"Results: {response['hits']['total']['value']}")
    
    if response['hits']['total']['value'] > 0:
        print("\n✓ QUERY_STRING WORKS! Use this approach.")
        
        # Display results
        table_data = []
        for i, hit in enumerate(response['hits']['hits'][:10], 1):
            source = hit['_source']
            row = {
                "#": i,
                "Timestamp": source.get('@timestamp', 'N/A'),
                "Host": source.get('host', {}).get('name', 'N/A'),
                "Service": source.get('service', {}).get('name', 'N/A'),
                "Account": source.get('winlog', {}).get('event_data', {}).get('TargetUserName', 'N/A'),
                "Ticket Options": source.get('winlog', {}).get('event_data', {}).get('TicketOptions', 'N/A')
            }
            table_data.append(row)
        
        print("\n" + tabulate(table_data, headers="keys", tablefmt="grid"))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()
finally:
    if 'client' in locals():
        client.close()
        print("\n✓ Connection closed")

✓ Successfully connected to Elasticsearch
✓ Cluster: elastic-container-project
✓ Version: 9.2.2

STEP 1: Testing basic query for event.code:4769
Results with event.code as string '4769': 91
Results with event.code as integer 4769: 91

✓ Found Event 4769 records! Here's a sample:

--- SAMPLE RECORD STRUCTURE ---
input.type: winlog
event.code: 4769
winlog.event_data.TicketEncryptionType: 0x12
winlog.event_data.TicketOptions: 0x40810000
service.name: 2016-LAB-DC$

--- FULL WINLOG EVENT DATA ---
{
  "Status": "0x0",
  "LogonGuid": "{82D1B86D-B40E-1117-E517-299C15C75D24}",
  "TicketEncryptionType": "0x12",
  "ServiceName": "2016-LAB-DC$",
  "TicketOptionsDescription": [
    "Forwardable",
    "Renewable",
    "Name-canonicalize"
  ],
  "StatusDescription": "KDC_ERR_NONE",
  "TicketOptions": "0x40810000",
  "ServiceSid": "S-1-5-21-4268768631-214544434-2094132827-1000",
  "TargetUserName": "2016-LAB-DC$@HACKLAB.COM",
  "TargetDomainName": "HACKLAB.COM",
  "TicketEncryptionTypeDescription": "A